In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import tensorflow as tf
# import os
# import matplotlib.pyplot as plt
# from tensorflow import keras 
# from keras.utils import to_categorical
# import matplotlib.pyplot as plt
# import sys
# import cv2 as cv
# from sklearn import model_selection
# from tqdm import tqdm
# import git 
# import pydicom
# from keras import metrics
# from tensorflow.keras.applications.xception import preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# tf.keras.backend.set_floatx('float16')
# models = tf.keras.models
# layers = tf.keras.layers
# utils = tf.keras.utils
# losses = tf.keras.losses
# optimizers = tf.keras.optimizers 
# metrics = tf.keras.metrics
# preprocessing_image = tf.keras.preprocessing.image
# applications = tf.keras.applications

# base_model = applications.Xception(include_top=True,
#                                    weights = None,
#                                    input_shape=(299,299,3),
#                                    pooling='avg',
#                                    classes = 4)

# def L2M_regularizer(x):
#     return (0.03 * tf.math.reduce_mean(x)) + (0.07*tf.reduce_sum(tf.square(x)))

# #regularizer = tf.keras.regularizers.l2(l2=0.01)


# for layer in base_model.layers:
#     if (hasattr(layer,'activations'))==True:
#          layer.activation = swish

# for layer in base_model.layers:
#     for attr in ['kernel_regularizer']:
#         if hasattr(layer, attr):
#             setattr(layer, attr, L2M_regularizer)
#     for attr in ['bias_regularizer']:
#         if hasattr(layer,attr):
#             setattr(layer, attr, L2M_regularizer)
#     for attr in ['activity_regularizer']:
#         if hasattr(layer,attr):
#             setattr(layer, attr, L2M_regularizer)

# opt = keras.optimizers.Adam(learning_rate=2e-4, epsilon = 1e-7, amsgrad=True, clipnorm = 1)
# met= ['categorical_accuracy']
# l = keras.losses.CategoricalCrossentropy()
# base_model.compile(loss = l, optimizer=opt, metrics = met)

# base_model.summary()

# BASE_DIR = "/kaggle/input/notebook7e3db3ba5c/COVID_data/images"

# datagen = ImageDataGenerator(
#     preprocessing_function = preprocess_input,
#     validation_split = 0.1)

# train_datagen = datagen.flow_from_directory(
#     BASE_DIR,
#     target_size = (299,299),
#     subset = 'training'
#     )
# val_datagen = datagen.flow_from_directory(
#     BASE_DIR,
#     target_size = (299,299),
#     subset = 'validation'
#     )

# history = base_model.fit(
#     train_datagen,
#     steps_per_epoch = 100,
#     epochs=10,
#     validation_data=val_datagen,
#     validation_steps = 20)

# base_model.save('/kaggle/working/')

In [ ]:
import os
import pydicom
from pydicom import dcmread
import pandas as pd
import numpy as np
#import pylibjpeg
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
#import gdcm
import vtk
from vtk.util import numpy_support
from vtk.util.numpy_support import vtk_to_numpy
import numpy
import cv2 as cv
from matplotlib import pyplot as plt

reader = vtk.vtkDICOMImageReader()
OUTPUT_PATH = '/kaggle/working/'
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data'))
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data/images'))
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data/images/Negative for Pneumonia'))
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data/images/Typical Appearance'))
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data/images/Indeterminate Appearance'))
os.mkdir(os.path.join(OUTPUT_PATH, 'COVID_data/images/Atypical Appearance'))

def process_data(f_name, img_array, o_class):
    cv.imwrite(os.path.join('/kaggle/working/COVID_data/', f"images/{o_class}/{f_name}.png"), img_array)
    return

def get_img(path):
    reader.SetFileName(path)
    reader.Update()
    im = reader.GetOutput()
    nx, ny, nz= im.GetDimensions()
    pointData = im.GetPointData()
    arrayData = pointData.GetAbstractArray(0)
    ArrayDicom = np.rot90(vtk_to_numpy(arrayData).reshape((nx, ny, nz), order = 'F'))
    cl1 = cv.convertScaleAbs(ArrayDicom, alpha=(255.0/65535.0))
    cl1 = cv.resize(cl1, (299, 299))
#     cl1 = cv.GaussianBlur(cl1,(5,5),0)
#     cl1 = cv.Laplacian(cl1,cv.CV_16S, 3)
#     kernel = np.array([[0.0, -1.0, 0.0], [-1.0, 5.0, -1.0],[0.0, -1.0, 0.0]]) high pass filter
#     kernel = kernel/(np.sum(kernel) if np.sum(kernel)!=0 else 1)
#     cl1= cv.filter2D(cl1,-1,kernel)
#     cl1 = cv.convertScaleAbs(cl1)
    table = np.array([((i / 255.0) ** (1/3)) * 255 #gamma correction
        for i in np.arange(0, 256)]).astype("uint8")
    cl1 = cv.LUT(cl1, table)
    clahe = cv.createCLAHE(clipLimit=255.0, tileGridSize=(3,3))
    cl1 = clahe.apply(cl1)
    cl1 = cv.merge([cl1,cl1,cl1])
    cl1 = ~cl1
    #hist_full = cv.calcHist([cl1],[0],None,[256],[0,256])
    plt.imshow(cl1)
    #plt.xlim([0,256])
    plt.show()
    return cl1
    

classifier_metadata = pd.read_csv('/kaggle/input/siim-covid19-detection/train_study_level.csv')
for dirname, _, filenames in os.walk('/kaggle/input/siim-covid19-detection/train/'):
    for filename in filenames:
        study = [x for x in os.path.join(dirname, filename).split('/')]
        study = study[5]+'_study'
        row = classifier_metadata[classifier_metadata['id'] == study]
        if row['Negative for Pneumonia'].item() == 1: 
            study = 'Negative for Pneumonia'
        elif row['Typical Appearance'].item() == 1: 
            study = 'Typical Appearance'
        elif row['Indeterminate Appearance'].item() == 1: 
            study = 'Indeterminate Appearance'
        elif row['Atypical Appearance'].item() == 1: 
            study = 'Atypical Appearance'
        process_data(filename, get_img(os.path.join(dirname, filename)), study)